# RHEL AI / OpenShift AI — Quick Win Demo

## Fuel Efficiency Prediction for Car Carrier (Cegonha) Trucks
This notebook trains a simple regression model to predict **fuel consumption (L/100km)** based on historical trip telemetry.
Use it in **OpenShift AI** (Jupyter) and then export the model for serving via **KServe**.


In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("sample_trips.csv")
df.head()


### Feature Engineering & Target

In [ ]:

# Target: fuel consumption per 100km
df = df.copy()
df['fuel_l_per_100km'] = (df['fuel_liters'] / df['route_distance_km']) * 100

features = [
    'num_cars','load_weight_kg','route_distance_km','avg_speed_kmh',
    'elevation_gain_m','ambient_temp_c','rain_mm','wind_kmh'
]
target = 'fuel_l_per_100km'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

rf = RandomForestRegressor(n_estimators=300, random_state=7, n_jobs=-1)
rf.fit(X_train, y_train)

preds = rf.predict(X_test)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

mae, r2


### Feature Importance

In [ ]:

importances = rf.feature_importances_
fi = pd.Series(importances, index=features).sort_values(ascending=False)
fi


In [ ]:

# Plot importance
plt.figure(figsize=(6,4))
fi.sort_values().plot(kind='barh')
plt.title("Feature Importance (RandomForest)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


### Save Model

In [ ]:

# save model artifact
joblib.dump(rf, "model_random_forest.joblib")
print("Saved model to model_random_forest.joblib")


### Try a What-if Scenario (Planned Trip)

In [ ]:

planned_trip = pd.DataFrame([{
    'num_cars': 7,
    'load_weight_kg': 7 * 1450,  # 7 cars ~1450kg each
    'route_distance_km': 680,
    'avg_speed_kmh': 78,
    'elevation_gain_m': 550,
    'ambient_temp_c': 26,
    'rain_mm': 3.0,
    'wind_kmh': 12
}])
pred = rf.predict(planned_trip)[0]
print(f"Predicted consumption: {pred:.2f} L/100km, estimated total fuel: {pred * planned_trip['route_distance_km'][0] / 100:.1f} L")
